Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ydata_profiling
%matplotlib inline

Config

In [ ]:
# Add your input path
labeled_path = "... /Data/labels.parquet

Loading Dataset

In [ ]:
dataset = pd.read_parquet(labeled_path)
dataset.head()

In [ ]:
profile = ydata_profiling.ProfileReport(dataset)
profile.to_file('Summary.html')

information of locations with zero demand

In [ ]:
df2 = dataset[dataset['Demand'] == 0.0]
df2

In [ ]:
len(df3.index)

Important Locations

In [ ]:
df4 = dataset.groupby('Location')['Demand'].aggregate(['mean']).sort_values('mean', ascending=False)
dataset_important_location = df4[:49]
dataset_important_location

In [ ]:
list_important_location = dataset_important_location.index
list_important_location

In [ ]:
dataset_important_location = dataset[dataset['Location'].isin(list_important_location)]
dataset_important_location

Generate temporary predicted Data

In [ ]:
def generate_temporary_prediction(dataset):
    predicted_demand = dataset.Demand.shift(1)
    predicted_data = pd.DataFrame(dataset[['Location', 'Date']])
    predicted_data['Demand'] = predicted_demand
    return predicted_data

In [ ]:
predicted_data = generate_temporary_prediction(dataset_important_location)
predicted_data = predicted_data.dropna()
predicted_data.head()

calculate loss functions

In [ ]:
def calculate_loss_mape(actual_data, predicted_data):
    
    losses_df = actual_data[['Location','Date']].copy()
    losses_df['actual_demand'] = actual_data.Demand
    losses_df['predicted_demand'] = predicted_data.Demand
    losses_df = losses_df.dropna()

    
    losses_df['error'] = ((np.abs(losses_df['actual_demand']-losses_df['predicted_demand']))/
                          (losses_df['actual_demand']))*100

    loss_per_location = losses_df.groupby('Location').agg({'error':'mean'})
    loss_per_date = losses_df.groupby('Date').agg({'error':'mean'})

    return loss_per_location, loss_per_date


In [ ]:
loss_per_location, loss_per_date = calculate_loss_mape(dataset_important_location, predicted_data)
loss_per_location.head()

In [ ]:
loss_per_date.head()

In [ ]:
loss_per_location.error.min(), loss_per_location.error.max()

Plot histogram of errors

In [ ]:
# matplotlib histogram

plt.hist(loss_per_location['error'], color = 'blue', edgecolor = 'black',
           bins = int((loss_per_location.error.max()-loss_per_location.error.min())/5))





plt.title('Histogram of Errors')
plt.xlabel('Error')
plt.ylabel('Locations')

plt.show()

In [ ]:
# matplotlib histogram

plt.hist(loss_per_date['error'], color = 'blue', edgecolor = 'black',
           bins = int((loss_per_date.error.max()-loss_per_date.error.min())/5))





plt.title('Histogram of Errors')
plt.xlabel('Error')
plt.ylabel('Data')

plt.show()